<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-fundamentals/unit04-multilayer-networks/3_efficient_data_loaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unit 4.4: Defining Efficient Data Loaders

How to set up a `DataLoader` for a folder containing image files.

## Setup

In [ ]:
!pip install GitPython

In [4]:
import os
from git import Repo

if not os.path.exists("mnist-pngs"):
  Repo.clone_from("https://github.com/rasbt/mnist-pngs", "mnist-pngs")

In [5]:
import pandas as pd

## Data Preparation

- Here, we check the CSV files listing the image names and labels

In [6]:
df_train = pd.read_csv('mnist-pngs/train.csv')
df_train.head()

,filepath,label
0,train/0/16585.png,0
1,train/0/24537.png,0
2,train/0/25629.png,0
3,train/0/20751.png,0
4,train/0/34730.png,0


In [7]:
df_test = pd.read_csv('mnist-pngs/test.csv')
df_test.head()

,filepath,label
0,test/0/66062.png,0
1,test/0/64675.png,0
2,test/0/62204.png,0
3,test/0/60407.png,0
4,test/0/67368.png,0


## Creating a validation split

- MNIST doesn't come with a validation set partition, so we are creating it here from the training set, using 10% of the training data for validation.

In [8]:
df_train = pd.read_csv('mnist-pngs/train.csv')
df_train = df_train.sample(frac=1, random_state=123)

loc = round(df_train.shape[0]*0.9)
df_new_train = df_train.iloc[:loc]
df_new_val = df_train.iloc[loc:]

df_new_train.to_csv('mnist-pngs/new_train.csv', index=None)
df_new_val.to_csv('mnist-pngs/new_val.csv', index=None)

In [9]:
df_new_train.head()

,filepath,label
29561,train/4/38855.png,4
26640,train/4/17837.png,4
24498,train/3/7672.png,3
24594,train/3/17906.png,3
24249,train/3/41969.png,3


In [10]:
df_new_val.head()

,filepath,label
8023,train/1/9221.png,1
26302,train/4/32108.png,4
54489,train/9/7053.png,9
2712,train/0/32086.png,0
1463,train/0/12095.png,0
